In [9]:
import pandas as pd
from datasets import Dataset

In [10]:
df = pd.read_feather("./data/processed/stt/wav2vec/test/test.feather")

In [11]:
Dataset.from_pandas(df)

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 961
})

In [12]:
from transformers import EarlyStoppingCallback

In [ ]:
class CustomStopper(keras.callbacks.EarlyStopping):
    def __init__(self, monitor='val_loss',
             min_delta=0, patience=0, verbose=0, mode='auto', start_epoch = 100): # add argument for starting epoch
        super(CustomStopper, self).__init__()
        self.start_epoch = start_epoch

    def on_epoch_end(self, epoch, logs=None):
        if epoch > self.start_epoch:
            super().on_epoch_end(epoch, logs)

In [26]:
class EearlyStoppingCallbackAfterNumEpochs(EarlyStoppingCallback):
    "A callback that prints a message at the beginning of training"
    def __init__(self, start_epoch, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.start_epoch = start_epoch

    def on_epoch_end(self, epoch, logs=None):
        if epoch > self.start_epoch:
            super().on_epoch_end(epoch, logs)

In [23]:
es = EarlyStoppingCallback()
print(es.early_stopping_threshold)
print(es.early_stopping_patience)

0
0.0
1


In [34]:
es10 = EearlyStoppingCallbackAfterNumEpochs(4,)
es10.start_epoch
es10.early_stopping_threshold
es10.early_stopping_patience

1

In [53]:
# Defining parent class
class Parent():
      
    # Constructor
    def __init__(self):
        self.value = "Inside Parent"
          
    # Parent's show method
    def show(self, name):
        print(self.value)
        print(name)
          
# Defining child class
class Child(Parent):
      
    # Constructor
    def __init__(self):
        self.value = "Inside Child"
          
    # Child's show method
    def show(self, *args):
        super().show(*args)
        print(self.value)
    
p = Parent()        
c = Child()

In [55]:
print(p.show("tim"))
c.show("Hello")

Inside Parent
tim
None
Inside Child
Hello
Inside Child
